# Data Description
- Dataset from StatLib, used in the 1983 ASA Exposition. Originally 397 observations; 5 rows with missing horsepower removed.
- There are 392 rows.
- There are 9 variables:
    - mpg: Miles per gallon
    - cylinders: Number of cylinders (4 to 8)
    - displacement: Engine displacement (cu. inches)
    - horsepower: Engine horsepower
    - weight: Vehicle weight (lbs.)
    - acceleration: 0-60 mph time (sec.)
    - year: Model year
    - origin: Car origin (1. American, 2. European, 3. Japanese)
    - name: Vehicle name

# Load Packages and Data

In [4]:
import numpy as np, statsmodels.api as sm
from ISLP import load_data
from ISLP.models import (ModelSpec as MS, summarize, poly, sklearn_sm)
from sklearn.model_selection import train_test_split
from functools import partial
from sklearn.model_selection import \
     (train_test_split, cross_validate, KFold, ShuffleSplit)
from sklearn.base import clone

In [4]:
Auto = load_data('Auto')
Auto.origin = Auto.origin.astype('category')
Auto.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin
name,,,,,,,,
chevrolet chevelle malibu,18.0,8,307.0,130,3504,12.0,70,1
buick skylark 320,15.0,8,350.0,165,3693,11.5,70,1
plymouth satellite,18.0,8,318.0,150,3436,11.0,70,1
amc rebel sst,16.0,8,304.0,150,3433,12.0,70,1
ford torino,17.0,8,302.0,140,3449,10.5,70,1


In [5]:
Auto.describe().round(1)

,mpg,cylinders,displacement,horsepower,weight,acceleration,year
count,392.0,392.0,392.0,392.0,392.0,392.0,392.0
mean,23.4,5.5,194.4,104.5,2977.6,15.5,76.0
std,7.8,1.7,104.6,38.5,849.4,2.8,3.7
min,9.0,3.0,68.0,46.0,1613.0,8.0,70.0
25%,17.0,4.0,105.0,75.0,2225.2,13.8,73.0
50%,22.8,4.0,151.0,93.5,2803.5,15.5,76.0
75%,29.0,8.0,275.8,126.0,3614.8,17.0,79.0
max,46.6,8.0,455.0,230.0,5140.0,24.8,82.0


# The Validation Set Approach
- We use the validation set approach to estimate test error rates for fitting linear models on the `Auto` dataset.
- Using `train_test_split()`, we split the 392 observations into training and validation sets of 196 each with `test_size=196`.
- To ensure reproducibility, we set `random_state=0`.

In [6]:
Auto_train, Auto_valid = train_test_split(Auto, test_size=196, random_state=0)

In [7]:
hp_mm = MS(['horsepower'])
X_train = hp_mm.fit_transform(Auto_train)
y_train = Auto_train['mpg']
model = sm.OLS(y_train, X_train)
results = model.fit()

In [8]:
X_valid = hp_mm.transform(Auto_valid)
y_valid = Auto_valid['mpg']
valid_pred = results.predict(X_valid)
np.mean((y_valid - valid_pred)**2)

23.61661706966988

- Estimated validation MSE for linear regression: $23.62$.
- `evalMSE()` function estimates validation error for polynomial regressions using model string and training/test sets.

In [9]:
def evalMSE(terms, response, train, test):
   mm = MS(terms)
   X_train = mm.fit_transform(train)
   y_train = train[response]
   X_test = mm.transform(test)
   y_test = test[response]
   results = sm.OLS(y_train, X_train).fit()
   test_pred = results.predict(X_test)
   return np.mean((y_test - test_pred)**2)

In [10]:
MSE = np.zeros(3)
for idx, degree in enumerate(range(1, 4)):
    MSE[idx] = evalMSE([poly('horsepower', degree)], 'mpg',
                       Auto_train, Auto_valid)
MSE

array([23.61661707, 18.76303135, 18.79694163])

- Error rates are $23.62$, $18.76$, and $18.80$.
- Different training/validation splits may yield different validation errors.

In [11]:
Auto_train, Auto_valid = train_test_split(Auto, test_size=196, random_state=3)
MSE = np.zeros(3)
for idx, degree in enumerate(range(1, 4)):
    MSE[idx] = evalMSE([poly('horsepower', degree)], 'mpg',
                       Auto_train, Auto_valid)
MSE

array([20.75540796, 16.94510676, 16.97437833])

- Validation set error rates for models with linear, quadratic, and cubic terms are $20.76$, $16.95$, and $16.97$.
- These results confirm that a quadratic function of `horsepower` predicts `mpg` better than a linear one, with no improvement from a cubic function.

# Cross-Validation
- To cross-validate generalized linear models in Python, use `sklearn`, which has a different API than `statsmodels`. 

- Data scientists often face the challenge of linking functions for tasks A and B to compute B(A(D)). When A and B are incompatible, a *wrapper* is needed. 

- The `ISLP` package offers `sklearn_sm()`, a wrapper to use `sklearn` cross-validation with `statsmodels` models. 

- `sklearn_sm()` takes a `statsmodels` model as its first argument, with optional `model_str` for formulas and `model_args` for additional fitting arguments, like `{'family':sm.families.Binomial()}` for logistic regression.

In [15]:
hp_model = sklearn_sm(sm.OLS)
X, Y = MS(['horsepower']).fit_transform(Auto), Auto['mpg']
cv_results = cross_validate(hp_model, X, Y, cv=Auto.shape[0]) # LOOCV
np.mean(cv_results['test_score'])

24.231513517929226

In [16]:
hp_model = sklearn_sm(sm.OLS, MS(['horsepower']))
X, Y = Auto.drop(columns=['mpg']), Auto['mpg']
cv_results = cross_validate(hp_model, X, Y, cv=Auto.shape[0]) # LOOCV
np.mean(cv_results['test_score'])

24.231513517929226

- The `cross_validate()` function requires an object with `fit()`, `predict()`, and `score()` methods, along with feature array `X` and response `Y`. The `cv` argument specifies the type of cross-validation: an integer for $K$-fold or the number of observations for LOOCV. It returns a dictionary; here, the cross-validated test score (MSE) is 24.23.
- The process can be automated using a for loop to iteratively fit polynomial regressions from degrees 1 to 5. It calculates the cross-validation error for each degree and stores it in the vector cv_error. The variable 'd' represents the polynomial degree.

In [13]:
cv_error = np.zeros(5)
H = np.array(Auto['horsepower'])
M = sklearn_sm(sm.OLS)
for i, d in enumerate(range(1,6)):
    X = np.power.outer(H, np.arange(d+1))
    M_CV = cross_validate(M, X, Y, cv=Auto.shape[0])
    cv_error[i] = np.mean(M_CV['test_score'])
cv_error

array([24.23151352, 19.24821312, 19.33498406, 19.42443031, 19.03320428])

- The test MSE drops sharply from linear to quadratic fits but shows no improvement with higher-degree polynomials.
- The `np.power()` function's `outer()` method applies operations like `add()`, `min()`, or `power()`. It has two arrays as arguments, and then forms a larger array where the operation is applied to each pair of elements of the two arrays.

In [14]:
A = np.array([3, 5, 9])
np.power.outer(A, np.arange(2))

array([[1, 3],
       [1, 5],
       [1, 9]])

- In the CV above example, we used $K=n$; we can also use $K<n$, which is faster. Using `KFold()` with $K=10$ and `random_state`, we store CV errors for polynomial fits from degrees one to five.

In [15]:
cv_error = np.zeros(5)
cv = KFold(n_splits=10,
           shuffle=True,
           random_state=0) # use same splits for each degree
for i, d in enumerate(range(1,6)):
    X = np.power.outer(H, np.arange(d+1))
    M_CV = cross_validate(M, X, Y, cv=cv)
    cv_error[i] = np.mean(M_CV['test_score'])
cv_error

array([24.20766449, 19.18533142, 19.27626666, 19.47848404, 19.13722016])

- Computation time for $K$-fold is shorter than LOOCV, although traditionally, LOOCV should be faster. Quadratic fits still perform similarly to higher-degree polynomials.
- The `cross_validate()` function can use different splitting methods, like `ShuffleSplit()`, for validation set or K-fold cross-validation.

In [21]:
# Example data
X = [i for i in range(10)]
# KFold with 3 splits
kf = KFold(n_splits=3, shuffle=True, random_state=0)
# Splitting the data
for train_index, test_index in kf.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)

TRAIN: [0 1 3 5 6 7] TEST: [2 4 8 9]
TRAIN: [0 2 3 4 5 8 9] TEST: [1 6 7]
TRAIN: [1 2 4 6 7 8 9] TEST: [0 3 5]


In [20]:
# Example data
X = [i for i in range(10)]
# ShuffleSplit with 3 splits, 30% test size, and a fixed random state
ss = ShuffleSplit(n_splits=3, test_size=0.3, random_state=0)
# Splitting the data
for train_index, test_index in ss.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)

TRAIN: [9 1 6 7 3 0 5] TEST: [2 8 4]
TRAIN: [2 9 8 0 6 7 4] TEST: [3 5 1]
TRAIN: [4 5 1 0 6 9 7] TEST: [2 3 8]


In [16]:
validation = ShuffleSplit(n_splits=1,
                          test_size=196,
                          random_state=0)
results = cross_validate(hp_model,
                         Auto.drop(['mpg'], axis=1),
                         Auto['mpg'],
                         cv=validation)
results['test_score']

array([23.61661707])

One can estimate the variability in the test error by running the following:

In [17]:
validation = ShuffleSplit(n_splits=10,
                          test_size=196,
                          random_state=0)
results = cross_validate(hp_model,
                         Auto.drop(['mpg'], axis=1),
                         Auto['mpg'],
                         cv=validation)
results['test_score'].mean(), results['test_score'].std()

(23.802232661034164, 1.4218450941091847)

- The standard deviation here isn’t a valid estimate of the mean test score variability due to overlapping samples but indicates Monte Carlo variation from different random folds.

`ShuffleSplit(n_splits=10, test_size=196, random_state=0)`:  
   - Generates 10 random train/test splits.
   - Each test set has 196 samples.
   - Splits are independent.

`KFold(n_splits=10, shuffle=True, random_state=0)`:  
   - Divides data into 10 consecutive, shuffled folds.
   - Each fold serves as the test set once.
   - Splits are dependent.

Key Differences:  
`ShuffleSplit` creates random, independent splits, while `KFold` ensures each sample is tested exactly once, with shuffling before splits.

# The Bootstrap

## Estimating the Accuracy of a Linear Regression Model

In [1]:
def boot_SE(func, D, n=None, B=1000, seed=0):
    rng = np.random.default_rng(seed)
    # Initialize variables to accumulate the first and second moments
    first_, second_ = 0, 0
    # If n is not specified, use the number of rows in DataFrame D
    n = n or D.shape[0]
    # Perform B bootstrap iterations
    for _ in range(B):
        # Randomly sample n indices from D with replacement
        idx = rng.choice(D.index, n, replace=True)        
        # Apply the provided function to the sampled data
        value = func(D, idx)        
        # Accumulate the first and second moments
        first_ += value
        second_ += value**2    
    # Compute and return the bootstrap standard error
    return np.sqrt(second_ / B - (first_ / B)**2)

- The bootstrap method assesses variability in coefficient estimates and predictions. We'll use it to evaluate the variability of estimates for $\beta_0$ and $\beta_1$ in a linear regression model predicting `mpg` from `horsepower` in the `Auto` dataset and compare these with standard error formulas.

- To use `boot_SE()`, you need a function with a data frame `D` and indices `idx`. For bootstrapping a specific regression, we demonstrate this with simple steps.

- Create a `boot_OLS()` function for bootstrapping a regression model using a formula. Use `clone()` to copy the formula for refitting on new data, ensuring all features, like `poly()`, are re-evaluated on resampled data.

In [5]:
def boot_OLS(model_matrix, response, D, idx):
    # Subset the DataFrame D to include only the rows specified by idx
    D_ = D.loc[idx]    
    # Extract the response variable (dependent variable) from the subset DataFrame
    Y_ = D_[response]    
    # Clone the model matrix and fit_transform it on the subset DataFrame to get the design matrix (independent variables)
    X_ = clone(model_matrix).fit_transform(D_)
    return sm.OLS(Y_, X_).fit().params

- The first two arguments of `boot_SE()` should remain unchanged during bootstrapping. Use `partial()` from `functools` to freeze these arguments in `boot_OLS()`.

In [6]:
hp_func = partial(boot_OLS, model_matrix=MS(['horsepower']), response='mpg')

In [21]:
hp_func?

Signature:      hp_func(D, idx)
Call signature: hp_func(*args, **kwargs)
Type:           partial
String form:    functools.partial(<function boot_OLS at 0x000002090D3AB9C0>, ModelSpec(terms=['horsepower']), 'mpg')
File:           c:\users\tuand\appdata\local\programs\python\python312\lib\functools.py
Docstring:     
partial(func, *args, **keywords) - new function with partial application
of the given arguments and keywords.

- Typing `hp_func?` reveals it has two arguments, `D` and `idx`, making it perfect for `boot_SE()`.

- Use `hp_func()` to create bootstrap estimates for intercept and slope by sampling with replacement, demonstrated on 10 samples.

In [22]:
rng = np.random.default_rng(0)
np.array([hp_func(Auto, rng.choice(Auto.index, 392,
                     replace=True)) for _ in range(10)])

array([[39.12226577, -0.1555926 ],
       [37.18648613, -0.13915813],
       [37.46989244, -0.14112749],
       [38.56723252, -0.14830116],
       [38.95495707, -0.15315141],
       [39.12563927, -0.15261044],
       [38.45763251, -0.14767251],
       [38.43372587, -0.15019447],
       [37.87581142, -0.1409544 ],
       [37.95949036, -0.1451333 ]])

Next, we use the `boot_SE()` function to compute the standard
errors of 1,000 bootstrap estimates for the intercept and slope terms.

In [23]:
boot_SE(hp_func, Auto, B=1000, seed=10)

intercept     0.731176
horsepower    0.006092
dtype: float64

- The bootstrap estimates are 0.7311 for ${\rm SE}(\hat{\beta}_0)$ and 0.0061 for ${\rm SE}(\hat{\beta}_1)$. Standard formulas, available via `summarize()` from `ISLP.sm`, can also compute these standard errors.

In [24]:
hp_model.fit(Auto, Auto['mpg'])
summarize(hp_model.results_)['std err']

intercept     0.717
horsepower    0.006
Name: std err, dtype: float64

- The formula-based standard errors for $\hat{\beta}_0$ and $\hat{\beta}_1$ are 0.717 and 0.006, differing from bootstrap estimates. This isn't a problem with the bootstrap; it reveals the assumptions in standard formulas. They depend on $\sigma^2$, estimated using RSS, which may be inflated due to non-linearity. The standard formulas unrealistically assume fixed $x_i$. The bootstrap doesn't rely on these assumptions, likely offering more accurate estimates.

- Next, we compute bootstrap and standard estimates for a quadratic model, showing better alignment due to improved model fit.

In [25]:
quad_model = MS([poly('horsepower', 2, raw=True)])
quad_func = partial(boot_OLS, quad_model, 'mpg')
boot_SE(quad_func, Auto, B=1000)

intercept                                  1.538641
poly(horsepower, degree=2, raw=True)[0]    0.024696
poly(horsepower, degree=2, raw=True)[1]    0.000090
dtype: float64

We  compare the results to the standard errors computed using `sm.OLS()`.

In [26]:
M = sm.OLS(Auto['mpg'],
           quad_model.fit_transform(Auto))
summarize(M.fit())['std err']

intercept                                  1.800
poly(horsepower, degree=2, raw=True)[0]    0.031
poly(horsepower, degree=2, raw=True)[1]    0.000
Name: std err, dtype: float64